In [0]:
pip install kafka-python

Python interpreter will be restarted.
Requirement already satisfied: kafka-python in /local_disk0/.ephemeral_nfs/envs/pythonEnv-7340c948-2650-401e-a92c-a016d86e5376/lib/python3.7/site-packages (2.0.2)
Python interpreter will be restarted.

In [0]:
%scala
System.setProperty("ssl.ca.location","dbfs:/FileStore/tables/cloudkarafka.ca");

res0: String = dbfs:/FileStore/tables/cloudkarafka.ca

In [0]:
%scala
System.clearProperty("java.security.auth.login.config");
System.clearProperty("sasl.jaas.config");
System.setProperty("java.security.auth.login.config", "dbfs:/FileStore/tables/cloudkarafka.jaas");
System.setProperty("sasl.jaas.config", "dbfs:/FileStore/tables/cloudkarafka.jaas");
println("auth.lang:" + System.getProperty("java.security.auth.login.config"));
println("sasl.jaas:" + System.getProperty("sasl.jaas.config"));

spark.sparkContext.addFile("dbfs:/FileStore/tables/cloudkarafka.jaas")


auth.lang:dbfs:/FileStore/tables/cloudkarafka.jaas
sasl.jaas:dbfs:/FileStore/tables/cloudkarafka.jaas

In [0]:
topic = "yrmfqh3q-input"

print("====================== CSV to AVRO ======================")
filePath = "/FileStore/tables/customer_applications.csv"
read = spark.read.format('csv').options(header='true').options(inferSchema='true').load(filePath)
read.show()

from kafka import KafkaProducer

print("read-type:", type(read))

def publishToKafka2(rows):
  print("publishToKafka2:", type(rows), rows)
  producer = KafkaProducer(
    bootstrap_servers=['omnibus-01.srvs.cloudkafka.com:9094','omnibus-03.srvs.cloudkafka.com:9094','omnibus-02.srvs.cloudkafka.com:9094'],
    client_id= "Kafka-Producer",
    security_protocol="SASL_SSL",
    sasl_mechanism="SCRAM-SHA-256",
    sasl_plain_username="yrmfqh3q",
    sasl_plain_password="WfKJEYL_hwNUHVU8laSbj6gXkPIw_xuc")
  for row in rows:
    print("publishToKafka2::row:", type(row), row)
    #producer.send(topic, {"message", str(row.asDict())} )
    print("row:", type(row), row)
    producer.send(topic, key=b'message', value=bytes(str(row.asDict()), "utf-8"))
  producer.flush() #Flush after queuing up all the rows.

read.foreachPartition(publishToKafka2)

====================== CSV to AVRO ======================
+---+-------+------+------+
 id|cust_id|app_id|active|
+---+-------+------+------+
 1| 1| 1| 1|
 2| 1| 2| 1|
 2| 2| 1| 1|
 2| 2| 2| 1|
 3| 3| 1| 1|
 4| 3| 2| 0|
+---+-------+------+------+

====================== AVRO to KAFKA ======================
+---+-------+------+------+
 id|cust_id|app_id|active|
+---+-------+------+------+
 1| 1| 1| 1|
 2| 1| 2| 1|
 2| 2| 1| 1|
 2| 2| 2| 1|
 3| 3| 1| 1|
 4| 3| 2| 0|
+---+-------+------+------+

read-type: <class 'pyspark.sql.dataframe.DataFrame'>